In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data=pd.read_csv('C:/Users/hp/Downloads/Github/lung-cancer.csv',names=range(0,58))
lung=pd.DataFrame(data)

In [4]:
lung.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,1,0,3,0,?,0,2,2,2,1,...,2,2,2,2,1,1,1,2,2,NaN
1,1,0,3,3,1,0,3,1,3,1,...,2,2,2,2,2,2,1,2,2,NaN
2,1,0,3,3,2,0,3,3,3,1,...,2,2,2,2,2,2,2,1,2,NaN
3,1,0,2,3,2,1,3,3,3,1,...,2,2,2,2,2,2,2,2,2,NaN
4,1,0,3,2,1,1,3,3,3,2,...,2,2,2,2,2,2,1,2,2,NaN


In [5]:
lung.drop(57,axis=1,inplace=True)

In [6]:
lung[4].value_counts()

1    15
2    12
?     4
0     1
Name: 4, dtype: int64

In [7]:
lung[4]=lung[4].replace('?',3)

In [8]:
lung[4]=pd.to_numeric(lung[4])

In [9]:
lung[38].value_counts()

2    18
1    11
3     2
?     1
Name: 38, dtype: int64

In [10]:
lung[38].replace(['?',3],inplace=True)

In [11]:
lung[38]=lung[38].astype(int)

In [12]:
lung[38].dtypes

dtype('int32')

In [13]:
x=lung.iloc[:,1:]
y=lung.iloc[:,0]

In [14]:
x.skew()

1     5.656854
2    -0.076060
3    -0.818578
4     0.377450
5     1.021592
6    -0.317258
7    -0.212952
8    -0.855212
9     3.283157
10    0.982000
11    1.020376
12    0.845483
13    0.116250
14   -0.317258
15   -0.390348
16    1.505686
17   -2.380876
18   -2.380876
19    1.315641
20   -0.196939
21   -1.212283
22   -1.681134
23   -1.212283
24    1.738471
25    0.095544
26    0.116250
27    0.849385
28    0.026913
29   -0.197956
30    1.359999
31   -0.684394
32   -0.684394
33   -2.126866
34    0.498290
35    0.693421
36   -0.157939
37    0.509512
38    0.095544
39   -0.111604
40    0.000000
41    0.507452
42    0.214303
43    0.985388
44   -0.325290
45   -0.285911
46   -0.371443
47    3.795046
48    3.795046
49    0.000000
50    0.000000
51    0.000000
52   -0.751711
53   -0.751711
54   -0.264542
55   -1.681134
56   -1.021592
dtype: float64

In [15]:
for col in x.columns:
    if x[col].skew()>0.55:
        x[col]=np.log1p(x[col])
    if x[col].skew()<-0.55:
        x[col]=np.square(x[col])

In [16]:
from sklearn.decomposition import PCA
pca=PCA(n_components=20)
x=pca.fit_transform(x)

In [17]:
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
x=scale.fit_transform(x)

In [18]:
def random(alg):
    r_state=0
    for r_score in range(43,101):
        x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=r_score)
        alg.fit(x_train,y_train)
        pred=alg.predict(x_test)
        accuracy=accuracy_score(y_test,pred)
        if accuracy > r_state:
            r_state=accuracy
            final_r=r_score
    print(r_state,'at',final_r)

In [19]:
def model(alg):
    ml=alg
    ml.fit(x_train,y_train)
    pred=ml.predict(x_test)
    print(accuracy_score(y_test,pred))
    print(confusion_matrix(y_test,pred))
    print(classification_report(y_test,pred))

In [20]:
def cross_val(alg):
    score=cross_val_score(alg,x,y,cv=5)
    print(score)
    print(score.mean())
    print(score.std())

In [21]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
random(lr)

0.7272727272727273 at 65


In [22]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=65)
model(lr)

0.7272727272727273
[[2 2 0]
 [0 3 1]
 [0 0 3]]
              precision    recall  f1-score   support

           1       1.00      0.50      0.67         4
           2       0.60      0.75      0.67         4
           3       0.75      1.00      0.86         3

    accuracy                           0.73        11
   macro avg       0.78      0.75      0.73        11
weighted avg       0.79      0.73      0.72        11



In [23]:
cross_val(lr)

[0.42857143 0.14285714 0.42857143 0.33333333 0.4       ]
0.3466666666666667
0.10768224019125767


In [24]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier()
random(dtc)

0.8181818181818182 at 58


In [25]:
model(dtc)

0.2727272727272727
[[1 3 0]
 [0 1 3]
 [0 2 1]]
              precision    recall  f1-score   support

           1       1.00      0.25      0.40         4
           2       0.17      0.25      0.20         4
           3       0.25      0.33      0.29         3

    accuracy                           0.27        11
   macro avg       0.47      0.28      0.30        11
weighted avg       0.49      0.27      0.30        11



In [26]:
cross_val(dtc)

[0.57142857 0.71428571 0.42857143 0.66666667 0.2       ]
0.5161904761904762
0.18578997236718547


In [27]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
parameters={'n_neighbors':range(1,20)}
grid=GridSearchCV(knn,parameters,cv=5)
grid.fit(x,y)
grid.best_params_

{'n_neighbors': 2}

In [28]:
knn=KNeighborsClassifier(n_neighbors=2)
random(knn)

0.7272727272727273 at 83


In [29]:
model(knn)

0.45454545454545453
[[2 2 0]
 [0 3 1]
 [1 2 0]]
              precision    recall  f1-score   support

           1       0.67      0.50      0.57         4
           2       0.43      0.75      0.55         4
           3       0.00      0.00      0.00         3

    accuracy                           0.45        11
   macro avg       0.37      0.42      0.37        11
weighted avg       0.40      0.45      0.41        11



In [30]:
cross_val(knn)

[0.28571429 0.28571429 0.71428571 0.33333333 0.4       ]
0.4038095238095238
0.16079169211737357


In [31]:
from sklearn.svm import SVC
svm=SVC()
parameters={'kernel':['rbf','linear'],'C':[1,10,100]}
grid=GridSearchCV(svm,parameters,cv=5)
grid.fit(x,y)
grid.best_params_

{'C': 1, 'kernel': 'rbf'}

In [32]:
svm=SVC(kernel='rbf')
random(svm)

0.6363636363636364 at 94


In [33]:
model(svm)

0.36363636363636365
[[0 4 0]
 [0 4 0]
 [0 3 0]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.36      1.00      0.53         4
           3       0.00      0.00      0.00         3

    accuracy                           0.36        11
   macro avg       0.12      0.33      0.18        11
weighted avg       0.13      0.36      0.19        11



In [34]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
parameters={'n_estimators':[10,100,500,1000]}
gr=GridSearchCV(rf,parameters,cv=5)
gr.fit(x,y)
print(gr.best_score_)
print(gr.best_params_)

0.5
{'n_estimators': 10}


In [35]:
rf=RandomForestClassifier(n_estimators=10)
random(rf)

0.7272727272727273 at 82


In [36]:
model(rf)

0.18181818181818182
[[1 2 1]
 [2 1 1]
 [0 3 0]]
              precision    recall  f1-score   support

           1       0.33      0.25      0.29         4
           2       0.17      0.25      0.20         4
           3       0.00      0.00      0.00         3

    accuracy                           0.18        11
   macro avg       0.17      0.17      0.16        11
weighted avg       0.18      0.18      0.18        11



In [37]:
from sklearn.externals import joblib
joblib.dump(lr,'lr_lung_cancer.pkl')

['lr_lung_cancer.pkl']